In [139]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import copy
import os
import os
import re
import string
import pandas as pd
import numpy as np
import gensim
import nltk
# nltk.download()
# nltk.download("stopwords")
from nltk.corpus import stopwords
from gensim.models.doc2vec import LabeledSentence
from gensim.models import word2vec
from gensim import utils
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from __future__ import print_function
from keras.models import Sequential  
from keras.layers import Dense, Activation,Dropout


In [130]:
file = open('./train.csv', 'r', encoding='utf-8')
res_file = open('./test.csv', 'r', encoding = 'utf-8')

In [131]:
return_df = pd.read_csv(res_file)
res_file.close()

In [256]:
# return_df

In [3]:
df = pd.read_csv(file, sep = ',')
file.close()


In [4]:
# res_df['Id']
df['new'] = df['Summary']+df['Text']
# df

In [5]:
df.fillna('!',inplace = True)

In [7]:
# df.dropna()
df_train = copy.deepcopy(df)

df_train = df_train[:460804]
df_train['Score'] -= 1
# df_train
df_test = copy.deepcopy(df)
df_test = df_test[460804:]
# df_test

In [82]:
reviews = df_train['new']
review_test = df_test['new']
res = df_train['Score']
res_test = df_test['Score']

In [9]:
# df_train.dropna()

In [257]:
from sklearn.model_selection import train_test_split


In [258]:
class datacleaning():
    def constructLabeledSentences(data):
        sentences=[]
        for index, row in data.iteritems():
            sentences.append(LabeledSentence(utils.to_unicode(row).split(), ['Text' + '_%s' % str(index)]))
        return sentences

    def textClean(text):
        text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
        text = text.lower().split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]    
        text = " ".join(text)
        return(text)

    def cleanup(text):
        text = datacleaning.textClean(text)
        text= text.translate(str.maketrans("","", string.punctuation))
        return text

In [259]:
allText = reviews.apply(datacleaning.cleanup)
allText_test = review_test.apply(datacleaning.cleanup)

In [260]:
sentences = datacleaning.constructLabeledSentences(allText)
sentence_test = datacleaning.constructLabeledSentences(allText_test)

/Users/ljxiao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [460]:
INPUT_DIMENSION=1000

In [461]:
class MySentences(object):
    """MySentences is a generator to produce a list of tokenized sentences 
    
    Takes a list of numpy arrays containing documents.
    
    Args:
        arrays: List of arrays, where each element in the array contains a document.
    """
    def __init__(self, *arrays):
        self.arrays = arrays
 
    def __iter__(self):
        for array in self.arrays:
            for document in array:
                for sent in nltk.sent_tokenize(document):
                    yield nltk.word_tokenize(sent)

def get_word2vec(sentences, location):
    """Returns trained word2vec
    
    Args:
        sentences: iterator for sentences
        
        location (str): Path to save/load word2vec
    """
    if os.path.exists(location):
        print('Found {}'.format(location))
        model = gensim.models.Word2Vec.load(location)
        return model
    
    print('{} not found. training model'.format(location))
    model = gensim.models.Word2Vec(sentences, size=INPUT_DIMENSION, window=5, min_count=5, workers=4)
    print('Model done training. Saving to disk')
    model.save(location)
    return model

In [462]:
w2vec = get_word2vec(MySentences(allText),'w2vmodel')


w2vmodel not found. training model
Model done training. Saving to disk


In [463]:
w2vec_test = get_word2vec(MySentences(allText_test),'w2vmodel_test')


w2vmodel_test not found. training model
Model done training. Saving to disk


In [464]:
class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [465]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2vec)
mean_embedded = mean_embedding_vectorizer.fit_transform(allText)

/Users/ljxiao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).


In [466]:
mean_embedding_vectorizer_test = MeanEmbeddingVectorizer(w2vec_test)
mean_embedded_test = mean_embedding_vectorizer.fit_transform(allText_test)

/Users/ljxiao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).


In [467]:
mean_embedded


array([[-0.05243551, -0.02769762, -0.06115679, ..., -0.0159556 ,
         0.12447834,  0.03116012],
       [-0.27977613, -0.24672899, -0.03321983, ..., -0.19077405,
         0.13434066, -0.04224192],
       [ 0.14574905, -0.19832307,  0.20233639, ...,  0.22789514,
        -0.27232435, -0.39158624],
       ...,
       [ 0.0545247 ,  0.02330562, -0.27810237, ...,  0.17907244,
         0.09386668, -0.04984564],
       [-0.08122585, -0.0351049 ,  0.18333995, ..., -0.13410072,
         0.10611135, -0.29965791],
       [-0.21933363,  0.30772182, -0.29920992, ..., -0.09799889,
         0.03371126,  0.30878082]])

In [468]:
mean_embedded_test

array([[-0.20045018, -0.21382578, -0.05998511, ...,  0.02013098,
         0.06724421, -0.0216706 ],
       [-0.04237926, -0.06556096, -0.0665714 , ...,  0.06260543,
         0.03746755,  0.03807595],
       [ 0.03357392, -0.27137223, -0.48566854, ..., -0.00579256,
         0.36872089,  0.03321708],
       ...,
       [-0.12596942, -0.02834934, -0.15631673, ...,  0.05561934,
         0.17696366, -0.0031377 ],
       [ 0.16402821,  0.30524489, -0.06436918, ...,  0.0514502 ,
        -0.00220442,  0.11592378],
       [-0.08391818,  0.03125412, -0.11925033, ..., -0.17400557,
         0.14263852, -0.01064425]])

In [469]:
np.save("mean_embedded.npy", mean_embedded)


In [470]:
np.save("mean_embedded_test.npy", mean_embedded_test)

In [471]:
arrays = np.load("mean_embedded.npy")

In [472]:
arrays_test = np.load("mean_embedded_test.npy")

In [473]:
X_train, X_test, Y_train, Y_test = train_test_split(arrays,res,test_size=0.1, random_state=20)

In [474]:
X_train.shape

(414723, 1000)

In [475]:
X_test.shape

(46081, 1000)

In [476]:
from keras.utils import to_categorical
y_train = to_categorical(Y_train, num_classes=5)
y_test = to_categorical(Y_test, num_classes=5)
y_train

array([[0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [477]:
y_train.shape

(414723, 5)

In [534]:
model = Sequential()
model.add(Dense(512, input_dim=INPUT_DIMENSION,activation ="relu"))  #100
# model.add(Dropout(0.5))    #0.3
model.add(Dense(128,activation ="relu"))    #80
# model.add(Dropout(0.5))    #0.5
model.add(Dense(64,activation ="relu"))    #36
# model.add(Dropout(0.5))
model.add(Dense(36,activation ="relu"))
model.add(Dense(5,activation ="softmax"))   #6

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
# model.summary()


In [535]:
clf = model.fit(X_train, y_train,
                batch_size=2048,   #20
                epochs=30,
                verbose=1,
                validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 414723 samples, validate on 46081 samples
Epoch 1/30
414723/414723 [==============================] - 19s 45us/step - loss: 1.1678 - acc: 0.6321 - val_loss: 1.0620 - val_acc: 0.6368
Epoch 2/30
414723/414723 [==============================] - 18s 43us/step - loss: 0.9992 - acc: 0.6405 - val_loss: 0.9519 - val_acc: 0.6490
Epoch 3/30
414723/414723 [==============================] - 18s 43us/step - loss: 0.9196 - acc: 0.6670 - val_loss: 0.9008 - val_acc: 0.6748
Epoch 4/30
414723/414723 [==============================] - 16s 39us/step - loss: 0.8815 - acc: 0.6812 - val_loss: 0.8709 - val_acc: 0.6809
Epoch 5/30
414723/414723 [==============================] - 17s 40us/step - loss: 0.8558 - acc: 0.6872 - val_loss: 0.8486 - val_acc: 0.6859
Epoch 6/30
414723/414723 [==============================] - 18s 43us/step - loss: 0.8362 - acc: 0.6922 - val_loss: 0.8306 - val_acc: 0.6919
Epoch 7/30
414723/414723 [==============================] - 18s 44us/step - loss: 0.8203 - acc: 0.6970 - val_

In [536]:
a=model.predict(X_test)
pre=[]
for i in a:
    bb =i.tolist()
    pre.append(bb.index(max(bb)))
sum = 0
for j in range(len(pre)):
    if pre[j] == Y_test.values[j]:
        sum+=1

In [537]:
sum/len(pre)



0.7207308869165165

In [538]:
result_test = model.predict(arrays_test)

In [539]:
result_test

array([[9.0799391e-01, 5.2932505e-02, 1.4453160e-02, 5.5721523e-03,
        1.9048342e-02],
       [8.6871393e-02, 1.4496094e-01, 2.4393438e-01, 2.5009528e-01,
        2.7413800e-01],
       [3.4338798e-05, 5.1569956e-04, 1.4549427e-02, 3.1856743e-01,
        6.6633308e-01],
       ...,
       [2.2258326e-02, 1.6062679e-02, 2.2833383e-02, 8.5690252e-02,
        8.5315531e-01],
       [1.8802394e-04, 1.9296582e-04, 9.3547098e-04, 2.4127211e-02,
        9.7455639e-01],
       [1.6060630e-03, 6.0607528e-04, 1.7593731e-03, 2.3677042e-02,
        9.7235143e-01]], dtype=float32)

In [540]:
result_score = []
for i in result_test:
    tmp =i.tolist()
    result_score.append(tmp.index(max(tmp)))

In [541]:
len(result_score)

100000

In [542]:
for i in range(len(result_score)):
    result_score[i] += 1
# result_score

In [543]:
final_score = copy.deepcopy(result_score)

In [544]:
return_df['Score'] = final_score

In [545]:

return_df.to_csv("predictedScore.csv",index=False,sep=',')

In [ ]:
import sklearn
from sklearn.ensemble import RandomForestClassifier  


In [ ]:
clf = RandomForestClassifier(n_estimators=100,max_depth=16,verbose=1).fit(X_train, y_train)
yy = clf.predict(X_test)
print('Accuracy: {}'.format(accuracy_score(y_test, yy)))

In [ ]:
type(y_test)

In [ ]:
type(yy)

In [ ]:
clf1 = XGBClassifier(max_depth=5, learning_rate=0.05,n_estimators=100,verbose=1).fit(X_train, Y_train)

In [ ]:
yy1 = clf1.predict(X_test)


In [ ]:
yy1

In [ ]:
y_test

In [ ]:
type(Y_test)

In [ ]:
lst = []
for i in Y_test:
    lst.append(i)

In [ ]:
from numpy import array
ar = array(lst)

In [ ]:
ar

In [ ]:
yy1

In [ ]:
lst2 = []
for i in yy1:
    lst2.append(i)

In [ ]:
ar2 = array(lst2)

In [ ]:
print('Accuracy: {}'.format(accuracy_score(ar, ar2)))